<a href="https://colab.research.google.com/github/leohcar/P_IA025A_2022S1/blob/main/Proyecto_encoder_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder

In [1]:
nome = 'Carlos Leonardo Ancasi Hinostroza'

print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


# Importação das bibliotecas

In [2]:
import collections
import itertools
import functools
import math
import random

import torch
import torchvision
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from torchvision.datasets import MNIST

In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print('GPU available:', device)

GPU available: cuda


In [4]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

#Dataset e dataloader

In [5]:
!mkdir ./data

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize([224,224]),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
print(f'numero de imagenes {len(train_dataset)}')
img, target = train_dataset[0]
print(img.size())

numero de imagenes 50000
torch.Size([3, 224, 224])


In [7]:
train_size = 40000
val_size = 10000
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [8]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 800
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([50, 3, 224, 224])
Valores mínimo e máximo dos pixels:  tensor(-1.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


#Bloco Básico da Resnet

In [9]:
from collections import OrderedDict
from torch import nn
from torch import Tensor


class MyBlock(nn.Module):

    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:
        super(MyBlock, self).__init__()
        # Escreva seu código aqui.
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # Escreva seu código aqui.
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [10]:
class CNN_resnet(nn.Module):
    def __init__(self, image_channels, num_classes):

        super(CNN_resnet,self).__init__()

        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2)
        self.layer4 = self.__make_layer(256, 512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)
      
    def __make_layer(self, in_channels, out_channels, stride):

        identity_downsample = None
        
        if stride != 1:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(out_channels))
        return nn.Sequential(MyBlock(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride),
            MyBlock(out_channels, out_channels))
    
    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(x.shape[0], -1)
        out = self.fc(out)
        return out


In [11]:
model = CNN_resnet(3,100).to(device)

In [12]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 12605028


In [16]:
from tqdm import tqdm

n_epochs = 20
lr = 3e-4

model = CNN_resnet(3,100).to(device)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    return loss.item()

epochs = []
loss_history = []
loss_epoch_end = []
total_trained_samples = 0

with torch.no_grad():
    mean_accuracy = np.average([
        validation_step(val_input_ids.to(device), val_target_ids.to(device))
        for val_input_ids, val_target_ids in val_dataloader])
    print(f'Accuracy: {mean_accuracy}')
    
for i in range(n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss: {loss}')
    
    with torch.no_grad():
        mean_accuracy = np.average([
            validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader])
        print(f'Accuracy: {mean_accuracy}')


Accuracy: 4.605711863040924


Training: 100%|██████████| 800/800 [02:21<00:00,  5.66it/s]


Epoch: 0/19 Loss: 3.3548858165740967
Accuracy: 3.4699570190906526


Training: 100%|██████████| 800/800 [02:22<00:00,  5.63it/s]


Epoch: 1/19 Loss: 2.561497449874878
Accuracy: 2.618174580335617


...

Epoch: 17/19 Loss: 0.15581075847148895
Accuracy: 2.238164882659912


Training: 100%|██████████| 800/800 [02:22<00:00,  5.62it/s]


Epoch: 18/19 Loss: 0.15749391913414001
Accuracy: 2.372631229162216


Training: 100%|██████████| 800/800 [02:21<00:00,  5.64it/s]


Epoch: 19/19 Loss: 0.07073740661144257
Accuracy: 2.24537264585495
